<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import sqlalchemy
import pandas as pd

In [3]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 

In [4]:
connection = sqlalchemy.create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}")

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''SELECT
               COUNT(id) vacancies_count
               FROM public.vacancies
            '''

In [6]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
# текст запроса
query_3_2 = f'''SELECT
               COUNT(id) employers_count
               FROM public.employers
            '''

In [8]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
# текст запроса
query_3_3 = f'''SELECT
               COUNT(id) areas_count
               FROM public.areas
            '''

In [10]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
# текст запроса
query_3_4 = f'''SELECT
               COUNT(id) industries_count
               FROM public.industries
            '''

In [14]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,industries_count
0,294


***

In [ ]:
# выводы по предварительному анализу данных

- В представленной базе 49197 вакансий от 23501 работодателя, распологающихся в 1362 регионах и разделенных на 294 сферы деятельности. 
- На каждого работодателя, в среднем, приходится по 2 вакансии (2,09). Скорее всего, у крупных компаний значительно больше вакансий, чем 2. Можно предположить, что некоторые работодатели просто не ищут сотрудников или что выборка состоит из достаточно маленьких компаний.
- Всего в базе данных представлено 1362 территориальных области, это говорит о том, что исходя из общего списочного количества в качестве региона указываются не только города России или в базе также представлены иногородние и зарубежные вакансии. Также возможен вариант разного написания одной и той же территориальной области при обозначении региона.
- Сфер деятельности достаточно много (294), но не очень понятно, насколько они уникальны, т.е. есть ли сферы разные по названию, но очень похожие по содержанию.

Для корректного понимания данных необходим более глубокий анализ, т.к. средние значения плохо отображают реальность.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [39]:
# текст запроса
query_4_1 = f'''SELECT
               a.name area,
               COUNT(v.id) cnt
               FROM public.vacancies v
               JOIN public.areas a ON v.area_id = a.id
               GROUP BY 1
               ORDER BY 2 DESC
            '''

In [40]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [41]:
# текст запроса
query_4_2 = f'''SELECT
               COUNT(id)
               FROM public.vacancies 
               WHERE salary_from IS NOT NULL 
               OR salary_to IS NOT NULL
            '''

In [42]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [50]:
# текст запроса
query_4_3 = f'''SELECT
               ROUND(AVG(salary_from)) salary_from_avg,
               ROUND(AVG(salary_to)) salary_to_avg
               FROM public.vacancies
            '''

In [51]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,salary_from_avg,salary_to_avg
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [52]:
# текст запроса
query_4_4 = f'''SELECT
               schedule,
               employment,
               COUNT(id)
               FROM public.vacancies 
               GROUP BY 1, 2
               ORDER BY 3 DESC
            '''

In [53]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [54]:
# текст запроса
query_4_5 = f'''SELECT
               experience,
               COUNT(id)
               FROM public.vacancies
               GROUP BY 1
               ORDER BY 2
            '''

In [55]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [ ]:
# выводы по детальному анализу вакансий

- Количество вакансий больше в крупных городах и уменьшается к региональным.
Наибольшее число вакансий представлено в Москве ~11%. На 12 городов (из 769) приходится 48% всех вакансий (23616 из 49197).
- В половине вакансий указана предлагаемая зарплата (24073 из 49197).
- Около 70% вакансий предполагают полный рабочий день с полной занятостью (35367). Около 15% вакансий предполагают полный рабочий день удаленно (7802). И 15% другие виды занятости.
- В более ~70% вакансий требуется небольшой опыт работы или без опыта. Чаще всего ищут сотрудников с опытом работы от 1 до 3 лет (26152), далее идет группа вакансий с поиском сотрудников без опыта работы (7197). ~28% вакансий со средним опытом работы от 3 до 6 лет (14511). И лишь 2% вакансий с большим опытом. Реже всего ищут сотрудников с опытом работы более 6 лет (1337). Позволяет сделать вывод, что опытные работники не стремятся менять работу.
- Для вакансий без опыта работы предлагаемая зарплатная вилка, в среднем, составляет от 34699 до 56990, для вакансий с опытом работы более 6 лет вилка, в среднем, составляет от 150156 до 197819. Видна прямая зависимость между опытом работы и предлагаемой зарплатной вилкой.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [14]:
# текст запроса
query_5_1 = f'''(SELECT
               e.name,
               COUNT(v.employer_id)
               FROM public.employers e
               JOIN public.vacancies v ON e.id = v.employer_id
               GROUP BY 1
               ORDER BY 2 DESC
               LIMIT 1)
               UNION ALL
               (SELECT
               e.name,
               COUNT(v.employer_id)
               FROM public.employers e
               JOIN public.vacancies v ON e.id = v.employer_id
               GROUP BY 1
               ORDER BY 2 DESC
               OFFSET 4
               LIMIT 1)
            '''

In [15]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [12]:
# текст запроса
query_5_2 = f'''SELECT
               a.name,
               COUNT(e.id) employers_count,
               COUNT(v.id) vacancies_count
               FROM public.areas a
               LEFT JOIN public.employers e ON e.area = a.id
               LEFT JOIN public.vacancies v ON v.area_id = a.id
               WHERE v.id IS NULL
               GROUP BY 1
               ORDER BY 2 DESC
               LIMIT 1
            '''

In [13]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,name,employers_count,vacancies_count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [30]:
# текст запроса
query_5_3 = f'''SELECT
               e.name,
               COUNT(DISTINCT v.area_id)
               FROM public.employers e
               LEFT JOIN public.vacancies v ON v.employer_id = e.id
               GROUP BY e.id
               ORDER BY 2 DESC
            '''

In [31]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [32]:
# текст запроса
query_5_4 = f'''SELECT
               COUNT(e.id)
               FROM public.employers e
               LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
               WHERE ei.industry_id IS NULL
            '''

In [33]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [51]:
# текст запроса
query_5_5 = f'''SELECT
               e.name,
               COUNT(ei.industry_id)
               FROM public.employers e
               JOIN public.employers_industries ei ON ei.employer_id = e.id
               GROUP BY 1
               HAVING COUNT(ei.industry_id) = 4
               ORDER BY 1
               OFFSET 2
               LIMIT 1
            '''

In [52]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [57]:
# текст запроса
query_5_6 = f'''SELECT
               COUNT(e.id)
               FROM public.employers e
               JOIN public.employers_industries ei ON ei.employer_id = e.id
               JOIN public.industries i ON i.id = ei.industry_id
               WHERE i.name = 'Разработка программного обеспечения'
            '''

In [58]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [10]:
# код для получения списка городов-миллионников
# выгрузка списка городов-миллионников со страницы в Википедия
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
cities = []
table_cities = pd.read_html(url)[1]
for i in table_cities['Город']:
    if '[' in i: # Проверка на наличие сносок
        i = i[:-3]
    cities.append(i)
cities = tuple(cities)
cities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [9]:
# код для получения списка городов-миллионников
import requests 
from bs4 import BeautifulSoup

response = requests.get('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
page = BeautifulSoup(response.text, 'html.parser')
# Выбираем из документа вторую таблицу и в таблице выбираем первый уровень тега "a" внутри ячейки "td"
tda = page.select('div table:nth-of-type(2) tbody tr td > a')
cities = [t.text for t in tda]
cities = tuple(cities)
cities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [11]:
# текст запроса
query_5_7 = f'''(SELECT
               a.name,
               COUNT(v.id)
               FROM public.vacancies v 
               JOIN public.employers e ON e.id = v.employer_id
               JOIN public.areas a ON a.id = v.area_id 
               WHERE e.name = 'Яндекс' AND a.name IN {cities}
               GROUP BY 1)
               
               UNION ALL
               
               SELECT
               'Total',
               COUNT(v.id)
               FROM public.vacancies v 
               JOIN public.employers e ON e.id = v.employer_id
               JOIN public.areas a ON a.id = v.area_id 
               WHERE e.name = 'Яндекс' AND a.name IN {cities}
               
               ORDER BY 2
            '''

In [13]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [ ]:
# выводы по анализу работодателей

Ожидаемо, что топ ко количеству вакансий на ресурсе HH занимают крупные связанные с ИТ-сектором напрямую компании.
- Крупнейшими городами по числу вакансий и работодателей являются крупнейшие города России и ближайших стран СНГ: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы.
- Приблизительно у 36% работодателей (8419 из 23501) не указана сфера деятельности, но есть компании, которые связаны не менее чем с четырьмя сферами одновременно. Среди основных сфер деятельности указаны - "Разработка программного обеспечения" (3553 компании или около 15%) и "Системная интеграция", что в целом соответствует общей специфике рассматриваемой выборки.
- Крупным компаниям необходимы сотрудники в различных регионах. Работодатели по количеству регионов в которых представлены вакансии: Яндекс (181 регион), Ростелеком (152 региона), Спецремонт (116 регионов).
- Компания "Яндекс" является самой крупной по количеству вакансий (1933), затем идут: Ростелеком (491), Тинькофф (444), Сбер (428), Газпром нефть (331).
- Наибольшее количество работодателей находится в Москве, тут же открыто наибольшее количество вакансий. 
- Доля вакансий компании "Яндекс" в общей выборке невелика - около 4%, при этом достаточно равномерно распределена по городам (в т.ч. по городам-миллионникам - 485 вакансий или около 25%).

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [8]:
# текст запроса
query_6_1 = f'''SELECT
               COUNT(id)
               FROM public.vacancies
               WHERE LOWER(name) LIKE '%%data%%' 
               OR LOWER(name) LIKE '%%данн%%'
            '''

In [9]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [70]:
# текст запроса
query_6_2 = f'''SELECT
               COUNT(id)
               FROM public.vacancies
               WHERE (LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (LOWER(name) LIKE '%%ml%%' AND LOWER(name) NOT LIKE '%%html%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%')
               AND (name ILIKE '%%junior%%'
               OR experience = 'Нет опыта'
               OR employment = 'Стажировка')
            '''

In [73]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [75]:
# текст запроса
query_6_3 = f'''SELECT
               COUNT(id)
               FROM public.vacancies
               WHERE (LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%')
               AND (key_skills ILIKE '%%sql%%'
               OR key_skills ILIKE '%%postgres%%')
            '''

In [76]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [77]:
# текст запроса
query_6_4 = f'''SELECT
               COUNT(id)
               FROM public.vacancies
               WHERE (LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%')
               AND key_skills ILIKE '%%Python%%'
            '''

In [78]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [90]:
# Ключевые навыки разделены символом табуляции, можно вырезать этот символ и посчитать разницу 
# между исходной длиной строки и строкой с вырезанными символами табуляции, вычислив количество в строке 
# разделителей (символов табуляции). Количество ключевых навыков будет равно количеству разделителей (табуляций) + 1.

# текст запроса
query_6_5 = f'''SELECT
               ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, '\t', '')) + 1), 2)
               FROM public.vacancies
               WHERE LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%'
            '''

In [95]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [13]:
# текст запроса
query_6_6 = f'''SELECT
               experience,
               ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to)), 0)
               FROM public.vacancies
               WHERE LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%'
               GROUP BY 1
            '''

In [17]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,experience,round
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

In [ ]:
# выводы по предметному анализу

Из общей выборки 1771 вакансия (~3.5%) имеет отношение к данным, а непосредственно к DS - 480 (< 1%).

Для дата-сайентиста доступно от всех вакансий, связанных с данными:
- 51 вакансия (2,9%) для начинающих; 
- 201 вакансия (11%) со знанием SQL или Postgres; 
- 351 вакансия (20%) с владением Python.

Знание Python требуется чаще, чем знания SQL или Postgres. Также среди самых популярных навыков:
- Machine Learning; 
- Git; 
- Математическая статистика; 
- Pandas; 
- английский язык. 

Среднее количество требуемых ключевых навыков от специалистов по работе с данными для работодателей в вакансиях составляет 6,4 и незначительно отличается от количества в среднем по базе данных (6,37). 

На начальном этапе своей карьеры специалист DS может рассчитывать на среднюю зарплату около 75 тыс. рублей, чуть выше минимальной предлагаемой в среднем по рынку (74643 против 71065). 

С минимальным опытом, после года работы, показатель может увеличиться почти в два раза - до 140 тыс. рублей, значительно выше максимальной предлагаемой в среднем по рынку (139675 против 110537), тогда как после трех лет опыта зарплата может приблизиться к 250 тыс. рублей. 

Для специалистов с опытом более 6 лет зарплата вообще не указана. Вероятно, последнее объясняется слишком большой возможной разницей в вилке.

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

In [24]:
# 1.Зависимость средней зарплаты от типа рабочего графика для DS специалиста.
# текст запроса
query_7_1 = f'''SELECT
               schedule,
               ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to)), 0)::text salary_avg,
               COUNT(id)
               FROM public.vacancies
               WHERE (LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%')
               AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
               GROUP BY 1
               UNION ALL
               SELECT
               'Средняя зарплата по всем вакансиям DS',
               ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to)), 0)::text,
               COUNT(id)
               FROM public.vacancies
               WHERE (LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%')
               AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
               UNION ALL
               SELECT
               'Общее количество вакансий DS',
               ' ',
               COUNT(id)
               FROM public.vacancies
               WHERE LOWER(name) LIKE '%%data scientist%%' 
               OR LOWER(name) LIKE '%%data science%%'
               OR LOWER(name) LIKE '%%исследователь данных%%'
               OR (name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%')
               OR LOWER(name) LIKE '%%machine learning%%'
               OR LOWER(name) LIKE '%%машинн%%обучен%%'
               ORDER BY 3
            '''

In [25]:
# результат запроса
df = pd.read_sql_query(query_7_1, connection)
df

,schedule,salary_avg,count
0,Гибкий график,140500,4
1,Удаленная работа,190043,17
2,Полный день,175726,41
3,Средняя зарплата по всем вакансиям DS,177379,62
4,Общее количество вакансий DS,,480


Средняя зарплата при удаленной работе DS специалиста выше, чем в условиях полной занятости. Данный тип рабочего дня встречается реже классического полного дня, доля вакансий с графиком удаленной работы составляет 24% (117 вакансий) от общей выборки вакансий для DS. Как следствие, переезд в другой город для DS специалиста чаще всего не оправдан, т.к. возможно найти более высокооплачиваемую вакансию, предполагающую удаленный график работы. Также стоит обратить внимание, что для большей доли вакансий DS специалиста нет указаний по зарплате ни в каком виде, что в дальнейшем может значительно исказить качество модели по части параметров и факторов, привязанных к уровню заработной платы.

In [26]:
# 2.География вакансий DS, не имеющих график удаленной работы. Наиболее популярные регионы по количеству вакансий.
# текст запроса
query_7_2 = f'''SELECT
               a.name,
               COUNT(v.id)
               FROM public.vacancies v
               JOIN public.areas a ON a.id = v.area_id
               WHERE (LOWER(v.name) LIKE '%%data scientist%%' 
               OR LOWER(v.name) LIKE '%%data science%%'
               OR LOWER(v.name) LIKE '%%исследователь данных%%'
               OR (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%')
               OR LOWER(v.name) LIKE '%%machine learning%%'
               OR LOWER(v.name) LIKE '%%машинн%%обучен%%')
               AND v.schedule <> 'Удаленная работа'
               GROUP BY 1
               ORDER BY 2 DESC
               LIMIT 10
            '''

In [27]:
# результат запроса
df = pd.read_sql_query(query_7_2, connection)
df

,name,count
0,Москва,172
1,Санкт-Петербург,53
2,Новосибирск,18
3,Нижний Новгород,17
4,Алматы,12
5,Казань,11
6,Минск,8
7,Томск,7
8,Ростов-на-Дону,6
9,Нур-Султан,5


География вакансий DS специалиста, не имеющих график удаленной работы и подразумевающих присутствие на рабочем месте еще более сильно сконцентрирована в самых крупных городах, Москве и Санкт-Петербурге в сравнении со всей географией общей выборки. Учитывая, что стоимость жизни в данных городах заметно выше остальных городов общей выборки, и делая предположение, на основании предыдущего запроса query_7_1, о более высокой заработной плате при графике удаленной работы, также полагаем востребованность этого типа графика работы гораздо больше.

In [36]:
# 3.Наиболее популярные работодатели по количеству вакансий DS специалиста со всеми типами возможных графиков работы.
# текст запроса
query_7_3 = f'''SELECT
               e.name,
               COUNT(v.id)
               FROM public.vacancies v
               JOIN public.employers e ON e.id = v.employer_id
               WHERE LOWER(v.name) LIKE '%%data scientist%%' 
               OR LOWER(v.name) LIKE '%%data science%%'
               OR LOWER(v.name) LIKE '%%исследователь данных%%'
               OR (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%')
               OR LOWER(v.name) LIKE '%%machine learning%%'
               OR LOWER(v.name) LIKE '%%машинн%%обучен%%'
               GROUP BY 1
               ORDER BY 2 DESC
               LIMIT 10
            '''

In [38]:
# результат запроса
df = pd.read_sql_query(query_7_3, connection)
df

,name,count
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


Среди наиболее популярных работодателей по количеству вакансий DS специалиста со всеми типами возможных графиков работы выступают в основном крупнейшие российские ИТ и телекоммуникационные компании. Обращает на себя внимание тот факт, что крупнейший по количеству вакансий работодатель - компания "Яндекс", в общей выборке вакансий для DS занимает лишь 6 место. При этом в ТОП-3 входят два банка.

## Общий вывод по проекту
Исследование вакансий из HeadHunter позволило выявить крупнейших работодаталей и крупнейшие регионы по количеству вакансий связанных с it-сферой. Лидерами среди работодателей являются Яндекс, Тинькофф, Сбер и другие. Лидерами среди регионов являются Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Проведенный анализ позволил выявить среднии размеры предлагаемых заработных плат в зависимости от опыта кондидата, позволил оценить рынок и востребованность в специалистах дата-саентистов, очертить зарплатные ожидания дата-саентистов.

**Общие выводы:**

В рамках проекта мы рассмотрели выборку вакансий для IT-специалистов и набор сопутствующих баз данных для изучения и анализа этой информации с целью ее использования в будущем для создания модели машинного обучения, которая будет рекомендовать вакансии клиентам кадрового агентства, претендующим на позицию Data Scientist.

В рассматриваемых базах содержится информация о 49 тыс. вакансиях и 23,5 тыс. компаниях. Как изначально и предполагалось, далеко не по всем из указанным компаниям есть актуальные вакансии. В основном вакансии географически привязаны к Москве и Санкт-Петербурге, что в целом соответствует общей деловой активностью российской экономики.

Сразу бросается в глаза, тот факт, что более чем по половине вакансий отсутствует информация зарплате, что может негативно отразиться на качестве разрабатываемой модели, т.к. зарплатные ожидания один из основных факторов при принятии решения как для соискателя, так и для работодателя. При этом средняя нижняя граница равна 71 тыс. рублей, а средняя верхняя граница - 111 тыс. рублей.

Работодатели преимущественно ищут сотрудников на полную занятость, но часто допускают удаленную работу (вторая по популярности опция). При этом в выборке 83% приходится на вакансии, допускающим опыт менее 3 лет либо вообще отсутствие опыта.

Общая выборка практически не имеет какой-либо концентрации на работодателях. Доля крупнейшего работодателя - Яндекс - 4%, при этом география Яндекс гораздо более равномерна в сравнении с общей выборкой. Обращает на себя внимание отсутствие более чем у трети компаний информации о сфере деятельности. Среди наиболее распространенных - разработка программного обеспечения и системная интеграция, что в целом соответствует общей специфике рассматриваемой выборки.

Учитывая тот факт, что предполагается разработка специализированной модели, необходимо более детально рассмотреть целевую выборку. Из общей выборки 1771 вакансия (или 3,6%) имеет отношение к данным. Это говорит о том, что выявленные закономерности для всей выборки не могут быть экстраполированы на эту подвыборку и необходим детальный анализ таких вакансий. Для дата-сайентистов начального уровня доступно 51 вакансия или 2,9% от всех вакансий, связанных с данными. Среди ключевых навыков для DS указаны знание Python, умение работать с SQL или postgres.

Среди основных работодателей для дата-сайентистов выступают в основном крунейшие российские ИТ и телкоммуникационные компании. Обращает на себя тот факт, что крупнейший по количеству вакансий работодатель - Яндекс, в выборке вакансий для DS занимает лишь 6 место. При этом в ТОП-3 попали два банка. География подразумевающих посещение офиса вакансий для дата-сайентистов еще более сильно сконцентрирована на Москве и Санкт-Петербурге в сравнении с географией всей выборки.

Для выборки вакансий для DS, даже более чем для общей выборки, выражен фактор отсутствия данных об информации о зарплате. Доля таких вакансий в подвыборке составляет 87% против 51% по всей выборке. Тем не менее, если предположить, что тенденции, выявленные в тех вакансиям, где информация о зарплате указана, релевантны для всей подвыборки DS, то можно сделать следующие вывод:

- На начальном этапе своей карьеры дата-сайентист может рассчитывать на зарплату около 75 тыс. рублей, после года опыта показатель может увеличиться почти в два раза - до 140 тыс. рублей, тогда как после трех лет опыта зарплаты может приблизиться к 250 тыс. рублей. Это заметно выше аналогичных уровне для всей выборки - 40, 77 и 133 тыс. рублей соответственно.
- Средняя зарплата при удаленной работе для дата-сайентистов выше, чем при полном дне. Хотя такой тип рабочего дня распространен меньше классического полного дня, для вакансий, предполагающих удаленную работу составляют 24% (117 вакансий) от выборки вакансий для DS. В результате переезд в другой город для дата-сайентистов чаще всего не имеет смысла, т.к. можно найти более высокооплачиваемую вакансию, предполагающую удаленную работу.
- Принимая во внимание, что (a) стоимость жизни (living cost) в Москве и Санкт-Петербурге заметно выше других городов выборки и (b) зарплата при удаленной работе DS выше других вариантов графика работы, можно предположить повышенный спрос соискателей на удаленную работу.